## Multi-Step Task Decomposition with Recommendations
This tutorial demonstrates a complete workflow for LLM-powered task decomposition, plan generation, verification, refinement, and evaluation in the context of personalized home gym setup.

#### Objective
Use an LLM to decompose a user goal into a personalized, multi-step plan with recommendation, verification, refinement, and evaluation.

In [1]:
# %pip install -U langchain langchain-openai openai

# Install dependencies
# %pip install langchain openai --upgrade

##### Step 1: Prerequisites, user profile and tool function 

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json
import yaml
import os
# Read the YAML file
with open('./../../../Curify/curify_api.yaml', 'r') as yaml_file:
    data = yaml.safe_load(yaml_file)

# Access the API keys and other configuration data
openai_api_key = data.get('openai').get('api_key')
os.environ["OPENAI_API_KEY"] = openai_api_key
# OpenAI setup
# Create ChatOpenAI instance with key
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

In [3]:
# Step 1: User profile
user_profile = {
    "goal": "build a home gym",
    "budget": 2000,
    "space": "10 ft x 12 ft",
    "fitness_level": "intermediate",
    "preferences": ["cardio", "compact equipment"]
}

In [4]:
# Step 2: Tool functions
def verify_budget(equipment, budget_limit):
    total = sum(item["price"] for item in equipment)
    return {"within_budget": total <= budget_limit, "total": total}

def verify_space(equipment):
    compact_keywords = ["foldable", "adjustable", "resistance band"]
    compact_count = sum(any(kw in item["name"].lower() for kw in compact_keywords) for item in equipment)
    return {"space_ok": compact_count >= len(equipment) // 2, "compact_count": compact_count}

#### Step 2: Planner agent - generate structured JSON plan

In [7]:

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a planning assistant that returns JSON plans for home gym setup."),
    ("human", """Given the user's profile:
- Budget: ${budget}
- Room Size: {space}
- Fitness Level: {fitness_level}
- Preferences: {preferences}

Generate a JSON plan with:
- space_plan: string
- equipment: list of {{ "name": ..., "price": ... }}
- setup_notes: string""")
])

planner_input = planner_prompt.format_messages(**user_profile)
plan_response = llm(planner_input)
# Extract the actual JSON content from the code block
raw_text = plan_response.content.strip()

# Remove markdown-style code fences
if raw_text.startswith("```json"):
    raw_text = raw_text[len("```json"):].strip()
if raw_text.endswith("```"):
    raw_text = raw_text[:-3].strip()

# Now safely load
plan_json = json.loads(raw_text)
print("🔧 Generated Plan:\n", json.dumps(plan_json, indent=2))

🔧 Generated Plan:
 {
  "space_plan": "Arrange the equipment along the longer wall to maximize space. Leave a 3 ft wide path for movement and stretching. Use vertical storage solutions for smaller items.",
  "equipment": [
    {
      "name": "Folding Treadmill",
      "price": 600
    },
    {
      "name": "Compact Rowing Machine",
      "price": 500
    },
    {
      "name": "Adjustable Dumbbells Set",
      "price": 300
    },
    {
      "name": "Resistance Bands Set",
      "price": 50
    },
    {
      "name": "Wall-Mounted Pull-Up Bar",
      "price": 100
    },
    {
      "name": "Yoga Mat",
      "price": 30
    },
    {
      "name": "Compact Exercise Bike",
      "price": 400
    }
  ],
  "setup_notes": "Focus on compact and foldable equipment to save space. The folding treadmill and compact rowing machine can be stored upright when not in use. The adjustable dumbbells and resistance bands offer versatile strength training options without taking up much room. Ensure the p

#### Step 3: Tool execution

In [8]:

budget_result = verify_budget(plan_json["equipment"], user_profile["budget"])
space_result = verify_space(plan_json["equipment"])
print("💰 Budget Check:", budget_result)
print("📐 Space Check:", space_result)

💰 Budget Check: {'within_budget': True, 'total': 1980}
📐 Space Check: {'space_ok': False, 'compact_count': 2}


#### Step 4: Critic agent - evaluate plan

In [9]:
critic_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an evaluator that gives 1-10 scores for coherence, relevance, and personalization."),
    ("human", "Evaluate the following plan:\n\n{plan_json}")
])

critic_input = critic_prompt.format_messages(plan_json=json.dumps(plan_json))
critic_response = llm(critic_input)
print("📊 Critic Evaluation:\n", critic_response.content)

📊 Critic Evaluation:
 **Coherence: 9/10**

The plan is well-structured and provides a clear strategy for arranging and utilizing the space effectively. The focus on compact and foldable equipment is consistent throughout, and the use of vertical storage solutions is a logical approach to maximizing space. The setup notes complement the space plan by providing practical advice on equipment storage and usage.

**Relevance: 9/10**

The plan is highly relevant for someone looking to set up a home gym in a limited space. The selection of equipment is appropriate for a variety of workouts, including cardio, strength training, and flexibility exercises. The emphasis on compact and foldable equipment aligns well with the goal of maximizing space.

**Personalization: 7/10**

While the plan is generally applicable to anyone setting up a small home gym, it lacks specific details that would tailor it to an individual's unique needs or preferences. For example, it does not consider the user's fitne